In [1]:
import os
import time
import hashlib
import hmac
import base64
import uuid

from dotenv import load_dotenv
from pprint import pprint
import requests

In [2]:
load_dotenv('../src/.env')

True

In [3]:
secret = os.getenv('SWITCHBOT_SECRET')
token  = os.getenv('SWITCHBOT_TOKEN')

In [4]:
# Declare empty header dictionary
def make_api_header(secret, token):
    api_header = {}

    nonce = uuid.uuid4()
    t = int(round(time.time() * 1000))
    string_to_sign = '{}{}{}'.format(token, t, nonce)

    string_to_sign = bytes(string_to_sign, 'utf-8')
    secret = bytes(secret, 'utf-8')

    sign = base64.b64encode(hmac.new(secret, msg=string_to_sign, digestmod=hashlib.sha256).digest())
    # print ('Authorization: {}'.format(token))
    # print ('t: {}'.format(t))
    # print ('sign: {}'.format(str(sign, 'utf-8')))
    # print ('nonce: {}'.format(nonce))

    #Build api header JSON
    api_header['Authorization']=token
    api_header['Content-Type']='application/json'
    api_header['charset']='utf8'
    api_header['t']=str(t)
    api_header['sign']=str(sign, 'utf-8')
    api_header['nonce']=str(nonce)

    return api_header

In [5]:
host_domain = 'https://api.switch-bot.com'

デバイスIDの取得

In [ ]:
dev_list_url = f'{host_domain}/v1.1/devices'
api_header = make_api_header(secret, token)
response = (
    requests.get(
        url=dev_list_url,
        headers=api_header
    )
)
response.raise_for_status()
pprint(response.json())

In [7]:
device_list = response.json()['body']['deviceList']
for device in device_list:
    if device['deviceType'] == 'Smart Lock':
        device_id = device['deviceId']
        break

print(device_id)

E7891DA2EC42


ロックの状態取得

In [ ]:
dev_status_url = f'{host_domain}/v1.1/devices/{device_id}/status'
api_header = make_api_header(secret, token)
response = (
    requests.get(
        url=dev_status_url,
        headers=api_header
    )
)
response.raise_for_status()
pprint(response.json())

ロックの施錠と解錠

In [ ]:
dev_ctrl_url = f'{host_domain}/v1.1/devices/{device_id}/commands'
response = (
    requests.post(
        url=dev_ctrl_url,
        headers=api_header,
        json={
            'deviceType': 'Lock',
            'commandType': 'command',
            # 施錠するときはlock
            # 解錠するときはunlock
            'command': 'lock',
            'parameter': 'default'
        }
    )
)
response.raise_for_status()
pprint(response.json())